# Path Planning with PSO - Experiments

**IMPORTANTE**: Reinicie o kernel antes de executar (Kernel → Restart)

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path().resolve()
SCENARIOS_DIR = PROJECT_ROOT / "scenarios"
OUTPUT_DIR = PROJECT_ROOT / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Scenarios: {list(SCENARIOS_DIR.glob('scenario*.txt'))}")

In [ ]:
from src import (
    load_scenario,
    plot_scenario,
    PSOParams,
    pso_plan,
    estimate_waypoints,
    run_all_experiments,
    save_results_csv,
    print_results_summary,
)

print("Imports OK")

## 1. Verificar Estimativa de Waypoints

In [ ]:
print("Estimativa de waypoints por cenário:")
print("-" * 50)
for i in range(5):
    s = load_scenario(SCENARIOS_DIR / f"scenario{i}.txt")
    wp = estimate_waypoints(s)
    print(f"Scenario {i}: {len(s.obstacles):2d} obstáculos -> {wp} waypoints")

## 2. Teste Individual

In [4]:
# Testar cenário 1 (simples)
scenario = load_scenario(SCENARIOS_DIR / "scenario1.txt")
result = pso_plan(scenario, PSOParams(seed=42))

print(f"Converged: {result.converged}")
print(f"Waypoints: {result.n_waypoints}")
print(f"Particles: {result.n_particles}")
print(f"Path Length: {result.L:.2f}")
print(f"L_coll: {result.L_coll:.6f}")

if result.L_coll > 0:
    print("\n⚠️ AVISO: Caminho tem colisões!")
else:
    print("\n✅ Caminho livre de colisões")

In [ ]:
# Visualizar
plot_scenario(scenario, path=result.best_path, show=True)

## 3. Testar Todos os Cenários

In [ ]:
print("Testando todos os cenários...")
print("=" * 80)

for i in range(5):
    s = load_scenario(SCENARIOS_DIR / f"scenario{i}.txt")
    r = pso_plan(s, PSOParams(seed=42))
    
    status = "✅" if r.converged and r.L_coll == 0 else "❌"
    print(f"{status} Scenario {i}: Conv={r.converged}, WP={r.n_waypoints}, L={r.L:.1f}, L_coll={r.L_coll:.2f}")

## 4. Experimentos Completos com Plots

In [ ]:
params = PSOParams(seed=42)

# CONTROLE MANUAL DE WAYPOINTS (nome_arquivo -> n_waypoints)
# Adicione aqui os cenários que precisam de ajuste manual
manual_waypoints = {
    "scenario1.txt": 4,
    "scenario2.txt": 4,
}

print(f"Overrides manuais: {manual_waypoints}")

results = run_all_experiments(
    scenarios_dir=SCENARIOS_DIR,
    params=params,
    output_dir=OUTPUT_DIR,
    save_plots=True,
    verbose=True,
    manual_waypoints=manual_waypoints,
)

In [ ]:
# Salvar CSV
csv_file = OUTPUT_DIR / "pso_results.csv"
save_results_csv(results, csv_file)
print(f"Saved: {csv_file}")

In [ ]:
# Resumo
print_results_summary(results, params)

## 5. Visualizar Resultados

In [ ]:
from IPython.display import Image, display

for i in range(5):
    path_file = OUTPUT_DIR / f"scenario_{i}_pso_path.png"
    if path_file.exists():
        # Verificar se convergiu
        status = "✅" if results[i].converged else "❌ NÃO CONVERGIU"
        print(f"\n=== Scenario {i} {status} ===")
        display(Image(filename=str(path_file), width=600))

## 6. Análise do Cenário 4 (Difícil)

O cenário 4 é muito complexo e pode não ter solução viável com os recursos padrão.

In [ ]:
scenario4 = load_scenario(SCENARIOS_DIR / "scenario4.txt")

print(f"Cenário 4: {len(scenario4.obstacles)} obstáculos")
print(f"Start: ({scenario4.start1.x}, {scenario4.start1.y})")
print(f"Goal: ({scenario4.goal1.x}, {scenario4.goal1.y})")

# Visualizar sem caminho para ver os obstáculos
plot_scenario(scenario4, show=True)

In [ ]:
# Tentar com mais recursos
print("Tentando com mais recursos...")
result4 = pso_plan(scenario4, PSOParams(
    seed=42,
    max_iters=1000,
    max_restarts=10,
    n_particles=100,
))

print(f"Converged: {result4.converged}")
print(f"L_coll: {result4.L_coll:.2f}")
print(f"Restarts: {result4.restarts_used}")

if not result4.converged:
    print("\n⚠️ Este cenário pode não ter solução viável!")